# Lesson 3.  Dataset, Dataloader, BatchNorm, Dropout, Оптимизация

In [14]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.optim import Adam, RMSprop, SGD
from torch.utils.data import DataLoader, Dataset

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epoch_num = 10
lr = 10e-2

In [3]:
class California_Dataset(Dataset):
  def __init__(self, *init_datasets):
    assert all(init_datasets[0].size(0) == init_dataset.size(0) for init_dataset in init_datasets)
    self._base_datasets = init_datasets

  def __len__(self):
    return self._base_datasets[0].size(0)

  def __getitem__(self, idx):
      return tuple(base_dataset[idx] for base_dataset in self._base_datasets)

In [36]:
class Net(nn.Module):
  def __init__(self) -> None:
      super(Net, self).__init__()
      self.block_1 = nn.Sequential(
          nn.Linear(in_features=8, out_features=2**6, bias=True),
          nn.Dropout(0.1),
          nn.BatchNorm1d(2**6),
          nn.ReLU())
      self.block_2 = nn.Sequential(
          nn.Linear(in_features=2**6, out_features=2**5, bias=True),
          nn.Dropout(0.2),
          nn.BatchNorm1d(2**5),
          nn.ReLU())
      self.block_3 = nn.Sequential(
          nn.Linear(in_features=2**5, out_features=2**4, bias=True),
          nn.Dropout(0.2),
          nn.BatchNorm1d(2**4),
          nn.ReLU())
      self.block_4 = nn.Sequential(
          nn.Linear(in_features=2**4, out_features=2**2),
          nn.Dropout(0.2),
          nn.BatchNorm1d(2**2),
          nn.ReLU())
      self.predict = nn.Sequential(
          nn.Linear(in_features=2**2, out_features=1, bias=True),
          nn.BatchNorm1d(1),
          nn.ReLU())

  def forward(self, inp):
    out = self.block_1(inp)
    out = self.block_2(out)
    out = self.block_3(out)
    out = self.block_4(out)
    out = self.predict(out)
    return out[:, 0]

In [17]:
def train_loop(train_loader, test_loader, net, optimizer):
  loss_fn = nn.MSELoss()
  best_acc = {'train': None, 'test': None}
  net.train()
  for epoch in range(epoch_num):
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    for i, (inputs, labels) in enumerate(train_loader):

        outputs = net(inputs)
        loss = loss_fn(outputs, labels)

        # обнуляем градиент
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        running_items += len(labels)

        # выводим статистику о процессе обучения
        if i % 300 == 0 or (i + 1) == len(train_loader):    # печатаем каждые 300 mini-batches
            net.eval()

            test_loss, test_running_total, test_loss  = 0.0, 0.0, 0.0
            for y, (out_test, lbl_test) in enumerate(test_loader):
                test_outputs = net(out_test)
                test_loss += loss_fn(test_outputs, lbl_test)
                test_running_total += len(lbl_test)

            res_loss_train = running_loss / running_items
            res_loss_test = test_loss / test_running_total

            if best_acc['train'] is None or res_loss_train < best_acc['train']:
              best_acc['train'] = res_loss_train

            if best_acc['test'] is None or res_loss_test < best_acc['test']:
              best_acc['test'] = res_loss_train

            print(f'Epoch [{epoch + 1}/{epoch_num}]. ' \
                  f'Step [{i + 1}/{len(train_loader)}]. ' \
                  f'Loss: {res_loss_train:.3f}. '\
                  f'Test acc: {res_loss_test:.3f}.')

            running_loss, running_items = 0.0, 0.0
            net.train()
  print(f"Best acc train: {best_acc['train']:.3f}. Best acc test: {best_acc['test']:.3f}")
  print('Training is finished!')

In [29]:
california_housing = fetch_california_housing()
# Разделим на тестовые и тренеровочные данные
X_train, X_test, y_train, y_test = train_test_split(california_housing.data, california_housing.target, test_size=0.25, random_state=13)

In [30]:
# Нормализуем данные и подготовим их для дальнейшего использования в нашем dstaset
scale = StandardScaler()
X_train_s = scale.fit_transform(X_train)
X_test_s = scale.transform(X_test)

In [31]:
train_xt = torch.from_numpy(X_train_s.astype(np.float32)).to(DEVICE)
train_yt = torch.from_numpy(y_train.astype(np.float32)).to(DEVICE)

test_xt = torch.from_numpy(X_test_s.astype(np.float32)).to(DEVICE)
test_yt = torch.from_numpy(y_test.astype(np.float32)).to(DEVICE)

In [32]:
train_dataset = California_Dataset(train_xt, train_yt)
test_dataset = California_Dataset(test_xt, test_yt)

In [33]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, num_workers=2, drop_last=True)

# Adam

In [37]:
net = Net().to(DEVICE)
optimizer = Adam(net.parameters(), lr=lr)

In [38]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/10]. Step [1/241]. Loss: 0.069. Test acc: 0.065.
Epoch [1/10]. Step [241/241]. Loss: 0.014. Test acc: 0.008.
Epoch [2/10]. Step [1/241]. Loss: 0.012. Test acc: 0.008.
Epoch [2/10]. Step [241/241]. Loss: 0.009. Test acc: 0.008.
Epoch [3/10]. Step [1/241]. Loss: 0.007. Test acc: 0.007.
Epoch [3/10]. Step [241/241]. Loss: 0.009. Test acc: 0.017.
Epoch [4/10]. Step [1/241]. Loss: 0.011. Test acc: 0.015.
Epoch [4/10]. Step [241/241]. Loss: 0.008. Test acc: 0.007.
Epoch [5/10]. Step [1/241]. Loss: 0.014. Test acc: 0.012.
Epoch [5/10]. Step [241/241]. Loss: 0.008. Test acc: 0.010.
Epoch [6/10]. Step [1/241]. Loss: 0.012. Test acc: 0.009.
Epoch [6/10]. Step [241/241]. Loss: 0.008. Test acc: 0.011.
Epoch [7/10]. Step [1/241]. Loss: 0.008. Test acc: 0.011.
Epoch [7/10]. Step [241/241]. Loss: 0.009. Test acc: 0.008.
Epoch [8/10]. Step [1/241]. Loss: 0.014. Test acc: 0.008.
Epoch [8/10]. Step [241/241]. Loss: 0.008. Test acc: 0.006.
Epoch [9/10]. Step [1/241]. Loss: 0.007. Test acc: 0.006

# RMSProp

In [39]:
net = Net().to(DEVICE)
optimizer = RMSprop(net.parameters(), lr=lr)

In [40]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/10]. Step [1/241]. Loss: 0.100. Test acc: 10875010.000.
Epoch [1/10]. Step [241/241]. Loss: 0.011. Test acc: 0.006.
Epoch [2/10]. Step [1/241]. Loss: 0.008. Test acc: 0.007.
Epoch [2/10]. Step [241/241]. Loss: 0.009. Test acc: 0.009.
Epoch [3/10]. Step [1/241]. Loss: 0.011. Test acc: 0.007.
Epoch [3/10]. Step [241/241]. Loss: 0.009. Test acc: 0.012.
Epoch [4/10]. Step [1/241]. Loss: 0.008. Test acc: 0.010.
Epoch [4/10]. Step [241/241]. Loss: 0.008. Test acc: 0.010.
Epoch [5/10]. Step [1/241]. Loss: 0.007. Test acc: 0.008.
Epoch [5/10]. Step [241/241]. Loss: 0.008. Test acc: 0.008.
Epoch [6/10]. Step [1/241]. Loss: 0.009. Test acc: 0.008.
Epoch [6/10]. Step [241/241]. Loss: 0.008. Test acc: 0.006.
Epoch [7/10]. Step [1/241]. Loss: 0.010. Test acc: 0.006.
Epoch [7/10]. Step [241/241]. Loss: 0.008. Test acc: 0.010.
Epoch [8/10]. Step [1/241]. Loss: 0.007. Test acc: 0.010.
Epoch [8/10]. Step [241/241]. Loss: 0.008. Test acc: 0.009.
Epoch [9/10]. Step [1/241]. Loss: 0.006. Test acc

# SGD

In [41]:
net = Net().to(DEVICE)
optimizer = SGD(net.parameters(), lr=lr)

In [42]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/10]. Step [1/241]. Loss: 0.078. Test acc: 0.088.
Epoch [1/10]. Step [241/241]. Loss: 0.012. Test acc: 0.008.
Epoch [2/10]. Step [1/241]. Loss: 0.016. Test acc: 0.008.
Epoch [2/10]. Step [241/241]. Loss: 0.009. Test acc: 0.007.
Epoch [3/10]. Step [1/241]. Loss: 0.008. Test acc: 0.007.
Epoch [3/10]. Step [241/241]. Loss: 0.008. Test acc: 0.007.
Epoch [4/10]. Step [1/241]. Loss: 0.009. Test acc: 0.007.
Epoch [4/10]. Step [241/241]. Loss: 0.008. Test acc: 0.006.
Epoch [5/10]. Step [1/241]. Loss: 0.005. Test acc: 0.006.
Epoch [5/10]. Step [241/241]. Loss: 0.008. Test acc: 0.006.
Epoch [6/10]. Step [1/241]. Loss: 0.006. Test acc: 0.006.
Epoch [6/10]. Step [241/241]. Loss: 0.008. Test acc: 0.007.
Epoch [7/10]. Step [1/241]. Loss: 0.006. Test acc: 0.007.
Epoch [7/10]. Step [241/241]. Loss: 0.007. Test acc: 0.007.
Epoch [8/10]. Step [1/241]. Loss: 0.007. Test acc: 0.008.
Epoch [8/10]. Step [241/241]. Loss: 0.008. Test acc: 0.007.
Epoch [9/10]. Step [1/241]. Loss: 0.006. Test acc: 0.007

# SGD + Momentum

In [43]:
net = Net().to(DEVICE)
optimizer = SGD(net.parameters(), lr=lr, momentum=0.7)

In [44]:
%%time
train_loop(train_loader, test_loader, net, optimizer)

Epoch [1/10]. Step [1/241]. Loss: 0.056. Test acc: 0.077.
Epoch [1/10]. Step [241/241]. Loss: 0.011. Test acc: 0.008.
Epoch [2/10]. Step [1/241]. Loss: 0.010. Test acc: 0.008.
Epoch [2/10]. Step [241/241]. Loss: 0.009. Test acc: 0.008.
Epoch [3/10]. Step [1/241]. Loss: 0.008. Test acc: 0.007.
Epoch [3/10]. Step [241/241]. Loss: 0.008. Test acc: 0.007.
Epoch [4/10]. Step [1/241]. Loss: 0.008. Test acc: 0.006.
Epoch [4/10]. Step [241/241]. Loss: 0.008. Test acc: 0.007.
Epoch [5/10]. Step [1/241]. Loss: 0.004. Test acc: 0.007.
Epoch [5/10]. Step [241/241]. Loss: 0.008. Test acc: 0.006.
Epoch [6/10]. Step [1/241]. Loss: 0.007. Test acc: 0.007.
Epoch [6/10]. Step [241/241]. Loss: 0.008. Test acc: 0.010.
Epoch [7/10]. Step [1/241]. Loss: 0.006. Test acc: 0.010.
Epoch [7/10]. Step [241/241]. Loss: 0.008. Test acc: 0.006.
Epoch [8/10]. Step [1/241]. Loss: 0.006. Test acc: 0.006.
Epoch [8/10]. Step [241/241]. Loss: 0.008. Test acc: 0.012.
Epoch [9/10]. Step [1/241]. Loss: 0.009. Test acc: 0.011